In [23]:
import os
import random
import numpy as np
from io import BytesIO

# Plotting and dealing with images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf

# Interactive widgets
from ipywidgets import widgets

In [26]:
BASE_DIR = r'C:\Users\toran\Documents\GitHub\ML_PadiCare\data'

train_dir = os.path.join(BASE_DIR, 'train')
test_dir = os.path.join(BASE_DIR, 'test')
validation_dir = os.path.join(BASE_DIR, 'val')
# Directory with training 
train_leafblight_dir = os.path.join(train_dir, 'bacterial_leaf_blight')
train_panicleblight_dir = os.path.join(train_dir, 'bacterial_panicle_blight')
train_brownspot_dir = os.path.join(train_dir, 'brown_spot')
train_sheathblight_dir = os.path.join(train_dir, 'rice_sheath_blight')
train_normal_dir = os.path.join(train_dir, 'normal')
# Directory with validation 
validation_leafblight_dir = os.path.join(validation_dir, 'bacterial_leaf_blight')
validation_panicleblight_dir = os.path.join(validation_dir, 'bacterial_panicle_blight')
validation_brownspot_dir = os.path.join(validation_dir, 'brown_spot')
validation_sheathblight_dir = os.path.join(validation_dir, 'rice_sheath_blight')
validation_normal_dir = os.path.join(validation_dir, 'normal')
# Directory with test 
test_leafblight_dir = os.path.join(test_dir, 'bacterial_leaf_blight')
test_panicleblight_dir = os.path.join(test_dir, 'bacterial_panicle_blight')
test_brownspot_dir = os.path.join(test_dir, 'brown_spot')
test_sheathblight_dir = os.path.join(test_dir, 'rice_sheath_blight')
test_normal_dir = os.path.join(test_dir, 'normal')
print(f"Contents of base directory: {os.listdir(BASE_DIR)}")
print(f"\nContents of train directory: {train_dir}")
print(f"\nContents of validation directory: {validation_dir}")
print(f"\nContents of test directory: {test_dir}")

Contents of base directory: ['test', 'train', 'val']

Contents of train directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\train

Contents of validation directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\val

Contents of test directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\test


In [27]:
train_leafblight_fnames = os.listdir(train_leafblight_dir)
train_panicleblight_fnames = os.listdir(train_panicleblight_dir )
train_brownspot_fnames = os.listdir(train_brownspot_dir)
train_sheathblight_fnames = os.listdir(train_sheathblight_dir )
train_normal_fnames = os.listdir(train_normal_dir)

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [29]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     9,470,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,496,613 (36.23 MB)

 Trainable params: 9,496,613 (36.23 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics = ['accuracy']
    )

In [34]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=20,
    label_mode='categorical'
    )

# Instantiate the Dataset object for the validation set
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(150, 150),
    batch_size=20,
    label_mode='categorical'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150, 150),
    batch_size=20,
    label_mode='categorical'
    )

Found 2281 files belonging to 5 classes.
Found 759 files belonging to 5 classes.
Found 757 files belonging to 5 classes.


In [39]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = train_dataset.cache().shuffle(SHUFFLE_BUFFER_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
validation_dataset_final = validation_dataset.cache().prefetch(PREFETCH_BUFFER_SIZE)
test_dataset_final = test_dataset.cache().prefetch(PREFETCH_BUFFER_SIZE)

In [40]:
history = model.fit(
    train_dataset_final,
    epochs=15,
    validation_data=validation_dataset_final,
    verbose=2
    )

Epoch 1/15
115/115 - 36s - 313ms/step - accuracy: 0.9882 - loss: 0.0745 - val_accuracy: 0.8906 - val_loss: 0.5661
Epoch 2/15
115/115 - 27s - 238ms/step - accuracy: 0.9934 - loss: 0.0393 - val_accuracy: 0.8986 - val_loss: 0.6352
Epoch 3/15
115/115 - 26s - 222ms/step - accuracy: 0.9925 - loss: 0.0332 - val_accuracy: 0.8933 - val_loss: 0.6709
Epoch 4/15
115/115 - 23s - 199ms/step - accuracy: 0.9882 - loss: 0.0765 - val_accuracy: 0.7773 - val_loss: 1.6161
Epoch 5/15
115/115 - 22s - 195ms/step - accuracy: 0.9956 - loss: 0.0274 - val_accuracy: 0.8959 - val_loss: 0.7064
Epoch 6/15
115/115 - 22s - 192ms/step - accuracy: 0.9912 - loss: 0.0445 - val_accuracy: 0.8920 - val_loss: 0.8136
Epoch 7/15
115/115 - 25s - 221ms/step - accuracy: 0.9912 - loss: 0.0343 - val_accuracy: 0.8617 - val_loss: 0.9241
Epoch 8/15
115/115 - 23s - 201ms/step - accuracy: 1.0000 - loss: 1.9516e-04 - val_accuracy: 0.8893 - val_loss: 0.8879
Epoch 9/15
115/115 - 23s - 199ms/step - accuracy: 0.9956 - loss: 0.0269 - val_accura

In [41]:
# Definisikan widget FileUpload
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)

# Output widget untuk menampilkan hasil prediksi
out = widgets.Output()
display(out)

# Fungsi untuk memproses dan memprediksi setiap file gambar
def file_predict(filename, file, out):
    """Fungsi untuk membuat prediksi dan mencetak hasilnya."""
    # Load gambar, ubah ukuran, dan ubah ke array
    image = tf.keras.utils.load_img(file, target_size=(150, 150))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalisasi

    # Prediksi menggunakan model
    predictions = model.predict(image, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Mapping label kelas
    class_names = [
        "Bacterial Leaf Blight",
        "Bacterial Panicle Blight",
        "Brown Spot",
        "Rice Sheath Blight",
        "Normal"
    ]
    predicted_label = class_names[predicted_class]

    # Tampilkan hasil prediksi
    with out:
        print(f"{filename} -> {predicted_label}")

# Fungsi untuk menangani perubahan pada uploader
def on_upload_change(change):
    """Fungsi untuk menangani file yang diunggah dan memproses prediksi."""
    # Bersihkan output sebelumnya
    out.clear_output()

    # Ambil file yang baru diunggah
    items = change.new
    for item in items:  # Loop jika ada lebih dari satu file
        file_jpgdata = BytesIO(item.content)
        file_predict(item.name, file_jpgdata, out)

# Hubungkan fungsi observasi ke uploader
uploader.observe(on_upload_change, names='value')


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Output()

In [ ]:
history = model.fit(
    train_dataset_final,
    epochs=15,
    validation_data=validation_dataset_final,
    verbose=2
    )

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])